In [10]:
HOT_DOG_TRAIN_DIR = "hot-dog-not-hot-dog/train/hot_dog"
NOT_HOT_DOG_TRAIN_DIR = "hot-dog-not-hot-dog/train/not_hot_dog"
HOT_DOG_TEST_DIR = "hot-dog-not-hot-dog/test/hot_dog"
NOT_HOT_DOG_TEST_DIR = "hot-dog-not-hot-dog/test/not_hot_dog"
TRAIN_DIR = "hot-dog-not-hot-dog/train"
TEST_DIR = "hot-dog-not-hot-dog/test"

IMAGE_SIZE = 50

In [11]:
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
from tensorflow import keras
import numpy
import matplotlib.pyplot as plt

In [12]:
print("Training Hot Dog images : ",len(os.listdir(HOT_DOG_TRAIN_DIR)))
print("Training Not Hot Dog images : ",len(os.listdir(NOT_HOT_DOG_TRAIN_DIR)))
print("Test Hot Dog images : ",len(os.listdir(HOT_DOG_TEST_DIR)))
print("Test Not Hot Dog images : ",len(os.listdir(NOT_HOT_DOG_TEST_DIR)))

Training Hot Dog images :  249
Training Not Hot Dog images :  249
Test Hot Dog images :  250
Test Not Hot Dog images :  250


In [13]:
IMAGE_SIZE = 160
BATCH_SIZE = 32

train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "binary"
)
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "binary"
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [14]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [15]:
base_model.trainable = False
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [16]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [17]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"])

In [18]:
len(model.trainable_variables)

2

In [19]:
epochs = 1
steps_per_epoch = train_generator.n 
validation_steps = validation_generator.n

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs = epochs,
                              workers=4,
                              validation_data = validation_generator,
                              validation_steps = validation_steps)

Epoch 1/1
497/498 [============================>.] - ETA: 2s - loss: 0.2495 - acc: 0.9023

StopIteration: [Errno 2] No such file or directory: 'hot-dog-not-hot-dog/test/hot_dog/815805.jpg'